## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Mataura,NZ,2021-04-23 20:51:01,-46.1927,168.8643,45.00,95,53,1.99,broken clouds
1,1,Khatanga,RU,2021-04-23 20:51:01,71.9667,102.5000,33.06,97,100,21.21,overcast clouds
2,2,Tiksi,RU,2021-04-23 20:51:01,71.6872,128.8694,4.64,98,97,7.18,overcast clouds
3,3,Maputo,MZ,2021-04-23 20:51:02,-25.9653,32.5892,64.00,88,75,4.61,broken clouds
4,4,Kodiak,US,2021-04-23 20:51:02,57.7900,-152.4072,39.20,87,90,9.22,light rain


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 85
What is the maximum temperature you would like for your trip? 90


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
23,23,Tessalit,ML,2021-04-23 20:51:07,20.1986,1.0114,88.75,6,0,7.34,clear sky
70,70,Mayor Pablo Lagerenza,PY,2021-04-23 20:51:21,-19.9309,-60.7718,88.92,42,73,12.88,broken clouds
112,112,Conceicao Do Araguaia,BR,2021-04-23 20:51:32,-8.2578,-49.2647,87.80,62,0,3.44,clear sky
138,138,The Valley,AI,2021-04-23 20:51:40,18.2170,-63.0578,86.00,65,20,12.66,few clouds
151,151,San Cristobal,VE,2021-04-23 20:51:44,7.7669,-72.2250,89.60,58,40,9.22,scattered clouds
175,175,Arlit,NE,2021-04-23 20:51:51,18.7369,7.3853,89.20,7,38,11.65,scattered clouds
189,189,Biu,NG,2021-04-23 20:51:55,10.6129,12.1946,87.26,27,83,4.43,broken clouds
215,215,Acapulco,MX,2021-04-23 20:52:04,16.8634,-99.8901,87.80,58,90,6.91,overcast clouds
246,246,Saint-Georges,GD,2021-04-23 20:50:49,12.0564,-61.7485,86.00,66,20,20.71,few clouds
264,264,La Asuncion,VE,2021-04-23 20:52:18,11.0333,-63.8628,87.80,58,10,24.16,clear sky


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                24
City                   24
Country                24
Date                   24
Lat                    24
Lng                    24
Max Temp               24
Humidity               24
Cloudiness             24
Wind Speed             24
Current Description    24
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
preferred_cities_df.count().dropna()
clean_df = preferred_cities_df

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
23,Tessalit,ML,88.75,clear sky,20.1986,1.0114,
70,Mayor Pablo Lagerenza,PY,88.92,broken clouds,-19.9309,-60.7718,
112,Conceicao Do Araguaia,BR,87.80,clear sky,-8.2578,-49.2647,
138,The Valley,AI,86.00,few clouds,18.2170,-63.0578,
151,San Cristobal,VE,89.60,scattered clouds,7.7669,-72.2250,
175,Arlit,NE,89.20,scattered clouds,18.7369,7.3853,
189,Biu,NG,87.26,broken clouds,10.6129,12.1946,
215,Acapulco,MX,87.80,overcast clouds,16.8634,-99.8901,
246,Saint-Georges,GD,86.00,few clouds,12.0564,-61.7485,
264,La Asuncion,VE,87.80,clear sky,11.0333,-63.8628,


In [8]:
# Set parameters to search for hotels with 5000 meters
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}
# Iterate through the DataFrame.
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"

    # Use the search term: "lodging" and our latitude and longitude.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # Make request and get the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
    # Grab the first hotel from the results and store the name.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")     

Hotel not found... skipping.
Hotel not found... skipping.


In [9]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df.count().dropna()
clean_hotel_df = hotel_df

In [10]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [11]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""
# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [12]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)

# Call the figure to plot the data.
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))